In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Dataset path

In [ ]:
data_path = "/content/drive/MyDrive/skill-dev-proj/101_ObjectCategories"

In [ ]:
import os

classes = os.listdir(data_path)
print("Number of classes:", len(classes))
print(classes)

Number of classes: 102
['accordion', 'BACKGROUND_Google', 'ant', 'anchor', 'barrel', 'bass', 'airplanes', 'beaver', 'bonsai', 'binocular', 'camera', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'cannon', 'cellphone', 'ceiling_fan', 'car_side', 'chair', 'chandelier', 'crocodile', 'cougar_body', 'cougar_face', 'dalmatian', 'crayfish', 'crab', 'crocodile_head', 'cup', 'dollar_bill', 'dolphin', 'electric_guitar', 'dragonfly', 'emu', 'elephant', 'ewer', 'Faces', 'euphonium', 'Faces_easy', 'ferry', 'flamingo_head', 'gerenuk', 'flamingo', 'grand_piano', 'gramophone', 'helicopter', 'garfield', 'hedgehog', 'hawksbill', 'headphone', 'ibis', 'inline_skate', 'laptop', 'ketch', 'kangaroo', 'lamp', 'joshua_tree', 'Leopards', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'metronome', 'menorah', 'minaret', 'octopus', 'Motorbikes', 'nautilus', 'okapi', 'pagoda', 'pigeon', 'pizza', 'panda', 'platypus', 'pyramid', 'revolver', 'rooster', 'saxophone', 'rhino', 'schooner', 'snoopy', 'scorpion', 'soc

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

data_path = "/content/drive/MyDrive/skill-dev-proj/101_ObjectCategories"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

full_dataset = datasets.ImageFolder(data_path, transform=transform)

print("Total images:", len(full_dataset))
print("Number of classes:", len(full_dataset.classes))

Total images: 9159
Number of classes: 102


#Split datasets

In [ ]:
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset,
    [train_size, val_size, test_size]
)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
#checking if loader works
images, labels = next(iter(train_loader))

print("Image batch shape:", images.shape)
print("Label batch shape:", labels.shape)
print("First 10 labels:", labels[:10])

Image batch shape: torch.Size([32, 3, 224, 224])
Label batch shape: torch.Size([32])
First 10 labels: tensor([ 6, 52, 83, 82, 27,  6, 62,  4, 22, 64])


#Densenet121

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model = models.densenet121(pretrained=True)
model = model.to(device)

True
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 30.8M/30.8M [00:00<00:00, 146MB/s]


In [ ]:
num_classes = len(full_dataset.classes)

model.classifier = nn.Linear(
    model.classifier.in_features,
    num_classes
)
print(num_classes)

102


In [ ]:
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
# checking for one batch

for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True


model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(
    model.classifier[1].in_features,
    num_classes
)
model = model.to(device)

# Freeze backbone
for param in model.features.parameters():
    param.requires_grad = False

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 216MB/s]


#Training Loop

In [ ]:
epochs = 10

best_val_acc = 0

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.classifier.parameters(),   # EfficientNet classifier
    lr=0.001
)

for epoch in range(epochs):

    print("=" * 50)
    print(f"Starting Epoch {epoch+1}/{epochs}")
    print("=" * 50)

    # ================= TRAIN =================
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    print("Training phase...")

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_acc = 100 * train_correct / train_total

    print("Training complete.")
    print("Running validation...")

    # ================= VALIDATION =================
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(
            model.state_dict(),
            "/content/drive/MyDrive/skill-dev-proj/best_efficientnet_b0.pth"
        )
        print("🔥 New best model saved!")

    print(f"Epoch [{epoch+1}/{epochs}] Results:")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Train Acc: {train_acc:.2f}%")
    print(f"Val Acc: {val_acc:.2f}%")
    print("-" * 50)

Starting Epoch 1/10
Training phase...
Training complete.
Running validation...
🔥 New best model saved!
Epoch [1/10] Results:
Train Loss: 385.5345
Train Acc: 65.86%
Val Acc: 87.33%
--------------------------------------------------
Starting Epoch 2/10
Training phase...
Training complete.
Running validation...
🔥 New best model saved!
Epoch [2/10] Results:
Train Loss: 118.1572
Train Acc: 89.77%
Val Acc: 89.95%
--------------------------------------------------
Starting Epoch 3/10
Training phase...
Training complete.
Running validation...
🔥 New best model saved!
Epoch [3/10] Results:
Train Loss: 73.3436
Train Acc: 93.12%
Val Acc: 90.46%
--------------------------------------------------
Starting Epoch 4/10
Training phase...
Training complete.
Running validation...
Epoch [4/10] Results:
Train Loss: 55.1955
Train Acc: 94.56%
Val Acc: 89.22%
--------------------------------------------------
Starting Epoch 5/10
Training phase...
Training complete.
Running validation...
🔥 New best model saved!

#Testing

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = len(full_dataset.classes)  # make sure dataset is loaded

# Recreate model architecture
model = models.efficientnet_b0(pretrained=False)

model.classifier[1] = nn.Linear(
    model.classifier[1].in_features,
    num_classes
)

model = model.to(device)

# Load saved weights
model.load_state_dict(
    torch.load(
        "/content/drive/MyDrive/skill-dev-proj/best_efficientnet_b0.pth",
        map_location=torch.device('cpu')
    )
)

model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100 * test_correct / test_total

print("=" * 50)
print(f"Final Test Accuracy: {test_acc:.2f}%")
print("=" * 50)

Final Test Accuracy: 97.02%


#Freezing Backbone

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.efficientnet_b0(pretrained=True)

num_classes = len(full_dataset.classes)

model.classifier[1] = nn.Linear(
    model.classifier[1].in_features,
    num_classes
)

model = model.to(device)

In [ ]:
# Freeze feature extractor
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

classifier.1.weight
classifier.1.bias


In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.classifier.parameters(),
    lr=0.001
)

In [ ]:
epochs = 10

best_val_acc = 0
import torch

print(torch.cuda.is_available())

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.classifier.parameters(),   # EfficientNet classifier
    lr=0.001
)

for epoch in range(epochs):

    print("=" * 50)
    print(f"Starting Epoch {epoch+1}/{epochs}")
    print("=" * 50)

    # ================= TRAIN =================
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    print("Training phase...")

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_acc = 100 * train_correct / train_total

    print("Training complete.")
    print("Running validation...")

    # ================= VALIDATION =================
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(
            model.state_dict(),
            "/content/drive/MyDrive/skill-dev-proj/best_efficientnet_b0.pth"
        )
        print("🔥 New best model saved!")

    print(f"Epoch [{epoch+1}/{epochs}] Results:")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Train Acc: {train_acc:.2f}%")
    print(f"Val Acc: {val_acc:.2f}%")
    print("-" * 50)

True
Starting Epoch 1/10
Training phase...
Training complete.
Running validation...
🔥 New best model saved!
Epoch [1/10] Results:
Train Loss: 379.5785
Train Acc: 66.70%
Val Acc: 85.14%
--------------------------------------------------
Starting Epoch 2/10
Training phase...
Training complete.
Running validation...
🔥 New best model saved!
Epoch [2/10] Results:
Train Loss: 114.6468
Train Acc: 90.30%
Val Acc: 88.27%
--------------------------------------------------
Starting Epoch 3/10
Training phase...
Training complete.
Running validation...
🔥 New best model saved!
Epoch [3/10] Results:
Train Loss: 73.0190
Train Acc: 93.23%
Val Acc: 89.22%
--------------------------------------------------
Starting Epoch 4/10
Training phase...
Training complete.
Running validation...
🔥 New best model saved!
Epoch [4/10] Results:
Train Loss: 54.7720
Train Acc: 94.65%
Val Acc: 89.80%
--------------------------------------------------
Starting Epoch 5/10
Training phase...
Training complete.
Running validati

#Testing

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = len(full_dataset.classes)

model = models.efficientnet_b0(pretrained=False)

model.classifier[1] = nn.Linear(
    model.classifier[1].in_features,
    num_classes
)

model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
model.load_state_dict(
    torch.load(
        "/content/drive/MyDrive/skill-dev-proj/best_efficientnet_b0.pth",
        map_location=device
    )
)

model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100 * test_correct / test_total

print("=" * 50)
print(f"Final Test Accuracy: {test_acc:.2f}%")
print("=" * 50)

Final Test Accuracy: 88.87%
